In [296]:
from configs import OCCUPATION_LIST
from itertools import combinations
import pandas as pd 
import random
import numpy as np
from scipy import interpolate
from scipy.optimize import bisect

def find_pse(occ):
    df = pd.read_feather(f'data/{occ}.feather')
    arr = np.array(df.changepoints)
    return bisect(interpolate.interp1d(df['rank'], arr-0.5), min(df['rank']), max(df['rank']))


In [298]:
occupation_combinations = list(combinations(OCCUPATION_LIST, 2))

summary = {
    "occupation":[],
    "ff_pse":[],
    "mm_pse":[],
    "fm_pse":[],
    "mf_pse":[]
}

for lhs, rhs in occupation_combinations:
    summary['occupation'].append(f"{lhs}_{rhs}")

    summary['ff_pse'].append(find_pse(f"female_{lhs}_female_{rhs}"))
    summary['mm_pse'].append(find_pse(f"male_{lhs}_male_{rhs}"))
    summary['fm_pse'].append(find_pse(f"female_{lhs}_male_{rhs}"))
    summary['mf_pse'].append(find_pse(f"male_{lhs}_female_{rhs}"))



In [299]:
pd.DataFrame(summary)

ValueError: arrays must all be same length